In [125]:
import json
import pandas as pd
import numpy as np
import datetime as dt

In [148]:
query='easter'

In [149]:
output=dict()

In [150]:
output['word']=query

In [151]:
# Open JSON <---- THIS WILL BE REPLACED WITH MONGO QUERY
with open('d3demo/json/'+query+'.json', "r") as read_file:
    data = json.load(read_file)

In [152]:
# Set columns for df
columns=['counts','rank','time']

In [153]:
# Build df
df=pd.DataFrame(data, columns=columns)

In [154]:
# Calculate min, max, and mean rank
output['maxrank']=int(min(df['rank'].values))
output['minrank']=int(max(df['rank'].values))
output['meanrank']=int(np.round(np.mean(df['rank'].values)))

In [155]:
# Pull out beginning part of date (YY-MM-DD)
df['time'] = [t[:10] for t in df['time']]
# Convert date to date object
df['time'] = [dt.datetime.strptime(t, '%Y-%m-%d').date() for t in df['time']]
# Index df by date
df.set_index('time',inplace=True)
# Find today's date
dt.datetime.today().date()
# How many days should we have in our dataset?
daterange=(dt.datetime.today().date()-df.index.min()).days
# Which days are we missing data for?
# Get a list of all days between today and the beginning of time...
missingdays=[(dt.datetime.today() - dt.timedelta(days=x+1)).date() for x in range(daterange)]
# ... then remove any days that we *do* have in our data from our list of missing dates:
for day in df.index:
    missingdays.remove(day)
# Fill any missing days with placeholder values
for day in missingdays:
    df.loc[day] = None
    df.loc[day]['counts'] = int(0)
    df.loc[day]['rank'] = int(999999999)
# Sort days
df.sort_values(by='time',ascending=False,inplace=True)

In [156]:
# Convert time back to a string
df.index=[t.strftime("%Y-%m-%d") for t in df.index]

In [157]:
# Send dates, ranks and counts as arrays to the output dict
output['times']=df.index.values.tolist()
output['ranks']=[int(r) for r in df['rank'].values] # Convert from int64 to Python integers
output['counts']=[int(c) for c in df['counts'].values] # Convert from int64 to Python integers

In [158]:
with open('d3demo/json_from_api/'+query+'.json', 'w') as outfile:
    json.dump(output, outfile)